<a href="https://colab.research.google.com/github/TobiasMaugus/Pratica1-PM/blob/master/problema3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GCC118 - Programação Matemática
## Universidade Federal de Lavras
### Instituto de Ciências Exatas e Tecnológicas
#### Profa. Andreza C. Beezão Moreira (DMM/UFLA)
#### Prof. Mayron César O. Moreira (DCC/UFLA)

## Problema

A cidade de União Paulista enfrenta uma séria carência orçamentária. Em busca de uma solução de longo prazo, a câmara de vereadores da cidade aprovou uma melhoria da base de cobrança de impostos, que prevê a condenação de uma área habitacional no centro da cidade e sua substituição por um conjunto habitacional moderno.

O projeto envolve duas fases:


1.   Demolição das casas que estão aquém do padrão para liberar terreno para o novo projeto;
2.   Construção do novo conjunto urbano.

A seguir, daremos um resumo da situação:

Um total de 300 casas aquém do padrão pode ser demolido. Cada casa ocupa um lote de 0,25 acres. O custo de demolição de uma casa condenada é de $2.000. Os tamanhos de lotes para domicílios (unidades) simples, duplos, triplos e quádruplos são de 0,18; 0,28; 0,4 e 0,5 acres, respectivamente. Ruas, espaços abertos e instalações públicas ocupam 15% da área disponível.

No novo conjunto habitacional, as unidades triplas e quádruplas devem representar no mínimo 25% do total. Unidades simples devem representar no mínimo 20% de todas as unidades, e unidades duplas, no mínimo 10%. O imposto cobrado por unidade para unidades simples, duplas, triplas e quádruplas é de $1.000, $1.900, $2.700 e $3.400, respectivamente.

O custo de construção por unidade domiciliar simples, dupla, tripla e quádrupla é de $50.000, $70.000, $130.000 e $160.000, respectivamente. O financiamento acordado com um banco local será de no máximo $15 milhões.

Quantas unidades de cada tipo devem ser construídas para maximizar a arrecadação de impostos?


## Instalação da biblioteca PuLP

Para mais informações, acesse: [PuLP 2.9.0](https://pypi.org/project/PuLP/).

In [45]:
!pip install pulp
import pulp

## Declaração dos parâmetros

* $D$: Custo de demolição
* $A$: Área total em acres
* $U$: Área útil
* $F$: Financiamento máximo
* $O$: Orcamento para construção
* $N$: tipos de unidade =[simples, duplas, triplas, quádruplas].
* $r_i \in \mathbb{R}_+$:Imposto sobre cada unidade do tipo $i \in N$.  
* $a_i \in \mathbb{R}_+$: Área ocupada por unidade do tipo $i \in N$.      
* $c_i \in \mathbb{R}_+$:Custo de construção por unidade do tipo $i \in N$.

In [46]:
tipos_unidades = [1, 2, 3, 4]  # 1=simple, 2=dupla, 3=tripla, 4=quádrupla

# Impostos por tipo de unidade
impostos = {
    1: 1000,
    2: 1900,
    3: 2700,
    4: 3400
}

# Área ocupada por unidade (acres)
area_unidade = {
    1: 0.18,
    2: 0.28,
    3: 0.4,
    4: 0.5
}

# Custo de construção por tipo
custo_construcao = {
    1: 50000,
    2: 70000,
    3: 130000,
    4: 160000
}

# Custo de demolição
custo_demolicao_total = 300 * 2000

# Área total liberada
area_total_acres = 300 * 0.25
area_util = 0.85 * area_total_acres

# Financiamento máximo
orcamento_total = 15000000
orcamento_para_construcao = orcamento_total - custo_demolicao_total

## Declaração do objeto que representa o modelo matemático

In [47]:
modelo = pulp.LpProblem('habitacao_uniao_paulista', pulp.LpMaximize)

## Variáveis de decisão

* $x_i \ge 0$: quantidade de unidades do tipo  $i \in N$ a serem sonstruídas

In [48]:
x = pulp.LpVariable.dicts("x", tipos_unidades, lowBound=0, cat='Integer')

## Função objetivo

* Maximização da arrecadação de impostos: $\max 1000x_1 + 1900 x_2 + 2700x_3 + 3400X_4$

In [49]:
modelo += pulp.lpSum(impostos[i] * x[i] for i in tipos_unidades)

In [50]:
modelo

habitacao_uniao_paulista:
MAXIMIZE
1000*x_1 + 1900*x_2 + 2700*x_3 + 3400*x_4 + 0.0
VARIABLES
0 <= x_1 Integer
0 <= x_2 Integer
0 <= x_3 Integer
0 <= x_4 Integer

## Restrições



* Propoção de unidades a serem constuídas:

  * $x_1 \ge 0,2\sum_{i \in N} x_i$
  * $x_2 \ge 0,1\sum_{i \in N} x_i$
  * $x_3 + x_4 \ge 0,25\sum_{i \in N} x_i$

In [51]:
modelo += x[1] >= 0.2 * pulp.lpSum(x[i] for i in tipos_unidades)  # simples >= 20%
modelo += x[2] >= 0.1 * pulp.lpSum(x[i] for i in tipos_unidades)  # duplas >= 10%
modelo += x[3] + x[4] >= 0.25 * pulp.lpSum(x[i] for i in tipos_unidades)  # triplas+quádruplas >= 25%

* Área ocupada pelas unidades:
$\sum_{i \in N} x_ia_i \le U$

In [52]:
modelo += pulp.lpSum(area_unidade[i] * x[i] for i in tipos_unidades) <= area_util

* Orçamento toral de contrução:
$\sum_{i \in N} x_ic_i \le O$

In [53]:
modelo += pulp.lpSum(custo_construcao[i] * x[i] for i in tipos_unidades) <= orcamento_para_construcao

### Resolvendo o problema

In [54]:
status = modelo.solve()

## Imprimindo as soluções do problema

In [55]:
print('Status da solução:', pulp.LpStatus[status])
print('Valor ótimo (arrecadação de impostos): $', modelo.objective.value())

print('\nQuantidade ideal de unidades a construir:')
for i in tipos_unidades:
    tipo = ['Simples', 'Dupla', 'Tripla', 'Quádrupla'][i - 1]
    print(f"  {tipo}: {x[i].value():.2f} unidades")

Status da solução: Optimal
Valor ótimo (arrecadação de impostos): $ 341100.0

Quantidade ideal de unidades a construir:
  Simples: 35.00 unidades
  Dupla: 96.00 unidades
  Tripla: 37.00 unidades
  Quádrupla: 7.00 unidades
